In [ ]:
import os
import pandas as pd

RAW_PATH = os.path.expanduser("../data/Telematicsdata.csv")
df = pd.read_csv(RAW_PATH)

df.columns = df.columns.str.lower()

df["timestamp"] = pd.to_datetime(df["timemili"], unit="ms").dt.tz_localize("UTC")

In [ ]:
print(df["variable"].value_counts())

In [ ]:
def tidy_events(raw):
    out = []

    for _, row in raw.iterrows():
        var = row["variable"]
        val = row["value"]

        if var == "POSITION":
            lat, lon, alt = map(float, val.split(","))
            out.append({
                "device_id": row["deviceid"],
                "timestamp": row["timestamp"],
                "lat": lat,
                "lon": lon,
                "alt": alt,
                "event": "position"
            })

        elif var in {"INTERNAL BATTERY", "EXTERNAL BATTERY"}:
            out.append({
                "device_id": row["deviceid"],
                "timestamp": row["timestamp"],
                var.lower().replace(" ", "_"): float(val)
            })

        elif var in {"IGNITION_STATUS", "TOWING"}:
            out.append({
                "device_id": row["deviceid"],
                "timestamp": row["timestamp"],
                var.lower().replace(" ", "_"): int(val in {"true", "on", "1"})
            })

    return pd.DataFrame(out)

clean = tidy_events(df)

In [ ]:
# A "trip" == consecutive records with ignition ON, grouped by device & date
clean["ignition_on"] = clean.get("ignition_status", 0)     # col may be NaN for rows w/o that variable
clean["date"] = clean["timestamp"].dt.date
clean["trip_id"] = (clean["ignition_on"].diff().fillna(0) == 1).cumsum()

# Harsh-brake proxy: rapid drop in speed (if you later add speed data) or towing alarm
clean["harsh_flag"] = clean.get("towing", 0)


In [ ]:
import sqlite3

con = sqlite3.connect("drivedash.db")      # sits in your project root
cur = con.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS raw_events (
        device_id      TEXT,
        timestamp      TEXT,
        variable       TEXT,
        value          TEXT
    );
""")

cur.execute("""
    INSERT INTO raw_events VALUES ('DEV123', '2025-05-24T18:00:00Z', 'BATTERY', '95.0')
""")
con.commit()

print(cur.execute("SELECT * FROM raw_events").fetchall())

con.close()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("sqlite:///drivedash.db", echo=False)   # ← 3 slashes = relative path

# df is your cleaned DataFrame from Telematicsdata.csv
df.to_sql("clean_events", engine, if_exists="replace", index=False)

# Read back with pure SQL
kpis = pd.read_sql("SELECT COUNT(*) AS rows, AVG(value) AS avg_val FROM clean_events", engine)
print(kpis)


In [ ]:
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib, pathlib

engine = create_engine("sqlite:///../drivedash.db")
kpi = pd.read_sql("SELECT * FROM trip_kpis", engine)

# Binary label: trips with many ignition records → proxy for “active trip”
y = (kpi["ignition_records"] >= 50).astype(int)
X = kpi[["total_records"]]          # keep it super-simple for now

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression().fit(X_tr, y_tr)
print(classification_report(y_te, clf.predict(X_te)))

# save to models/
pathlib.Path("../models").mkdir(exist_ok=True)
joblib.dump(clf, "../models/active_trip_lr.pkl")
